# Centering a trajectory in the box

Here we use MDAnalysis transformations to make a protein whole, center it in the box, and then wrap the water back into the box. We then look at how to do this on-the-fly.

**Last executed:** Feb 07, 2020 with MDAnalysis 0.20.2-dev0

**Last updated:** January 2020

**Minimum version of MDAnalysis:** 0.19.0

**Packages required:**
    
* MDAnalysis (<a data-cite="michaud-agrawal_mdanalysis_2011" href="https://doi.org/10.1002/jcc.21787">Michaud-Agrawal *et al.*, 2011</a>, <a data-cite="gowers_mdanalysis_2016" href="https://doi.org/10.25080/Majora-629e541a-00e">Gowers *et al.*, 2016</a>)
* MDAnalysisTests
   
**Optional packages for visualisation:**

* [nglview](http://nglviewer.org/nglview/latest/) (<a data-cite="nguyen_nglviewinteractive_2018" href="https://doi.org/10.1093/bioinformatics/btx789">Nguyen *et al.*, 2018</a>)





In [1]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import TPR, XTC
import numpy as np
import nglview as nv

_ColormakerRegistry()

## Loading files

The test files we will be working with here are trajectories of a adenylate kinase (AdK), a phosophotransferase enzyme. (<a data-cite="beckstein_zipping_2009" href="https://doi.org/10.1016/j.jmb.2009.09.009">Beckstein *et al.*, 2009</a>)

For the step-by-step transformations, we need to load the trajectory into memory so that our changes to the coordinates persist. If your trajectory is too large for that, see the [on-the-fly transformation](#Doing-all-this-on-the-fly) section for how to do this out-of-memory.

In [2]:
u = mda.Universe(TPR, XTC, in_memory=True)

## Before transformation

If you have NGLView installed, you can view the trajectory as it currently is.

In [ ]:
view = nv.show_mdanalysis(u)
view.add_representation('point', 'resname SOL')
view.center()
view

Otherwise, we embed a gif of it below.

![original](original.gif)

For easier analysis and nicer visualisation, we want to center this protein in the box.

## Unwrapping the protein

The first step is to "unwrap" the protein from the border of the box, to make the protein whole. MDAnalysis provides the `AtomGroup.unwrap` function to do this easily. Note that this function requires your universe to have bonds in it.

We loop over the trajectory to unwrap for each frame.

In [3]:
protein = u.select_atoms('protein')

for ts in u.trajectory:
    protein.unwrap(compound='fragments')

As you can see, the protein is now whole, but not centered.

In [ ]:
unwrapped = nv.show_mdanalysis(u)
unwrapped.add_representation('point', 'resname SOL')
unwrapped.center()
unwrapped

Over the course of the trajectory it leaves the box.

![unwrapped](unwrapped.gif)

## Centering in the box

The next step is to center the protein in the box. We calculate the center-of-mass of the protein and the center of the box for each timestep. We then move *all* the atoms so that the protein center-of-mass is in the center of the box.

If you don't have masses in your trajectory, try using the `center_of_geometry`.

In [4]:
for ts in u.trajectory:
    protein_center = protein.center_of_mass(pbc=True)
    dim = ts.triclinic_dimensions
    box_center = np.sum(dim, axis=0) / 2
    u.atoms.translate(box_center - protein_center)

The protein is now in the center of the box, but the solvent is likely outside it, as we have just moved all the atoms.

In [ ]:
centered = nv.show_mdanalysis(u)
centered.add_representation('point', 'resname SOL')
centered.center()
centered

![centered](centered.gif)

## Wrapping the solvent back into the box

Luckily, MDAnalysis also has `AtomGroup.wrap` to wrap molecules back into the box. Our trajectory has dimensions defined, which the function will find automatically. If your trajectory does not, or you wish to use a differently sized box, you can pass in a ``box`` with dimensions in the form ``[lx, ly, lz, alpha, beta, gamma]``.

In [5]:
not_protein = u.select_atoms('not protein')

for ts in u.trajectory:
    not_protein.wrap(compound='residues')

And now it is centered!

In [ ]:
wrapped = nv.show_mdanalysis(u)
wrapped.add_representation('point', 'resname SOL')
wrapped.center()
wrapped

![wrapped](wrapped.gif)

## Doing all this on-the-fly

Running all the transformations above can be difficult if your trajectory is large, or your computational resources are limited. Use on-the-fly transformations to keep your data out-of-memory.

Some common transformations are defined in `MDAnalysis.transformations`.

In [7]:
import MDAnalysis.transformations as trans

We re-load our universe.

In [8]:
u2 = mda.Universe(TPR, XTC)

protein2 = u2.select_atoms('protein')
not_protein2 = u2.select_atoms('not protein')

If you are using MDAnalysis from version 0.21.0 onwards, the `MDAnalysis.transformations` module contains `wrap` and `unwrap` functions. If not, we need to define a transformation ourselves.

A transformation is a function takes a `MDAnalysis.coordinates.base.Timestep` of a trajectory, modifies the `positions`, and returns the timestep.

In [9]:
def unwrap(ts):
    protein2.unwrap()
    return ts
    
def wrap(ts):
    not_protein2.wrap()
    return ts

If you need arguments besides the timestep, you can return a wrapped function. The outer function contains your other arguments. The inner `wrapped_function` function only takes the `ts` argument and returns it, as usual.

In [10]:
def unwrap_atomgroup(ag):
    def wrapped_function(ts):
        ag.unwrap()
        return ts
    return wrapped_function

def wrap_atomgroup(ag):
    def wrapped_function(ts):
        ag.wrap()
        return ts
    return wrapped_function

At this point, you can pass your transformations into your trajectory. You can only use `add_transformations` *once*, so pass them all in at once.

In [11]:
transforms = [unwrap_atomgroup(protein2),
              trans.center_in_box(protein2, wrap=True),
              wrap_atomgroup(not_protein2)]

u2.trajectory.add_transformations(*transforms)

In [ ]:
otf = nv.show_mdanalysis(u2)
otf.add_representation('point', 'resname SOL')
otf.center()
otf

![on the fly](on_the_fly.gif)

## References

[1] Richard&nbsp;J. Gowers, Max Linke, Jonathan Barnoud, Tyler J.&nbsp;E. Reddy, Manuel&nbsp;N. Melo, Sean&nbsp;L. Seyler, Jan Domański, David&nbsp;L. Dotson, Sébastien Buchoux, Ian&nbsp;M. Kenney, and Oliver Beckstein.
<span class="bibtex-protected">MDAnalysis</span>: <span class="bibtex-protected">A</span> <span class="bibtex-protected">Python</span> <span class="bibtex-protected">Package</span> for the <span class="bibtex-protected">Rapid</span> <span class="bibtex-protected">Analysis</span> of <span class="bibtex-protected">Molecular</span> <span class="bibtex-protected">Dynamics</span> <span class="bibtex-protected">Simulations</span>.
<em>Proceedings of the 15th Python in Science Conference</em>, pages 98–105, 2016.
00152.
URL: <a href="https://conference.scipy.org/proceedings/scipy2016/oliver_beckstein.html">https://conference.scipy.org/proceedings/scipy2016/oliver_beckstein.html</a>, <a href="https://doi.org/10.25080/Majora-629e541a-00e">doi:10.25080/Majora-629e541a-00e</a>.

[2] Naveen Michaud-Agrawal, Elizabeth&nbsp;J. Denning, Thomas&nbsp;B. Woolf, and Oliver Beckstein.
<span class="bibtex-protected">MDAnalysis</span>: <span class="bibtex-protected">A</span> toolkit for the analysis of molecular dynamics simulations.
<em>Journal of Computational Chemistry</em>, 32(10):2319–2327, July 2011.
00778.
URL: <a href="http://doi.wiley.com/10.1002/jcc.21787">http://doi.wiley.com/10.1002/jcc.21787</a>, <a href="https://doi.org/10.1002/jcc.21787">doi:10.1002/jcc.21787</a>.

[3] Hai Nguyen, David&nbsp;A Case, and Alexander&nbsp;S Rose.
<span class="bibtex-protected">NGLview</span>–interactive molecular graphics for <span class="bibtex-protected">Jupyter</span> notebooks.
<em>Bioinformatics</em>, 34(7):1241–1242, April 2018.
00024.
URL: <a href="https://academic.oup.com/bioinformatics/article/34/7/1241/4721781">https://academic.oup.com/bioinformatics/article/34/7/1241/4721781</a>, <a href="https://doi.org/10.1093/bioinformatics/btx789">doi:10.1093/bioinformatics/btx789</a>.

[4] Oliver Beckstein, Elizabeth&nbsp;J. Denning, Juan&nbsp;R. Perilla, and Thomas&nbsp;B. Woolf.
Zipping and <span class="bibtex-protected">Unzipping</span> of <span class="bibtex-protected">Adenylate</span> <span class="bibtex-protected">Kinase</span>: <span class="bibtex-protected">Atomistic</span> <span class="bibtex-protected">Insights</span> into the <span class="bibtex-protected">Ensemble</span> of <span class="bibtex-protected">Open</span>↔<span class="bibtex-protected">Closed</span> <span class="bibtex-protected">Transitions</span>.
<em>Journal of Molecular Biology</em>, 394(1):160–176, November 2009.
00107.
URL: <a href="https://linkinghub.elsevier.com/retrieve/pii/S0022283609011164">https://linkinghub.elsevier.com/retrieve/pii/S0022283609011164</a>, <a href="https://doi.org/10.1016/j.jmb.2009.09.009">doi:10.1016/j.jmb.2009.09.009</a>.